In [1]:
import pandas as pd
import os
import glob
import regex as rx

In [5]:
file = "csv/edrp_elec.csv"

In [6]:
df = pd.read_csv(file)

In [7]:
df

,ANON_ID,ADVANCEDATETIME,HH,ELECKWH
0,5110,15FEB08:12:30:00,25,0.600
1,11617,15FEB08:12:30:00,25,0.250
2,4869,15FEB08:12:30:00,25,0.300
3,9015,15FEB08:12:30:00,25,0.400
4,11628,15FEB08:12:30:00,25,0.850
...,...,...,...,...
413836033,12306,04JAN10:21:00:00,42,0.460
413836034,11620,14AUG08:19:30:00,39,0.450
413836035,1404,12JUL10:22:30:00,45,0.492
413836036,6552,13AUG09:04:30:00,9,0.014


In [9]:
df.ANON_ID.unique().shape

(14319,)

In [12]:
df.ADVANCEDATETIME.unique().shape

(47491,)

In [69]:
def format_date(row):
    months = {'JAN': '01', 'FEB': '02', 'MAR': '03', 'APR': '04', 'MAY': '05', 'JUN': '06',
          'JUL': '07', 'AUG': '08', 'SEP': '09', 'OCT': '10', 'NOV': '11', 'DEC': '12'}
    mon = rx.findall("[A-Z]+", row)[0]
    row = row.replace(mon, f"-{months[mon]}-")
    row = '20' + row[6:8] + row[2:6] + row[:2] + " " + row[9:]
    return row
    

In [73]:
df.ADVANCEDATETIME = df.ADVANCEDATETIME.apply(format_date)

In [74]:
df.ADVANCEDATETIME = pd.to_datetime(df.ADVANCEDATETIME)
df

,ANON_ID,ADVANCEDATETIME,HH,ELECKWH
0,5110,2008-02-15 12:30:00,25,0.600
1,11617,2008-02-15 12:30:00,25,0.250
2,4869,2008-02-15 12:30:00,25,0.300
3,9015,2008-02-15 12:30:00,25,0.400
4,11628,2008-02-15 12:30:00,25,0.850
...,...,...,...,...
413836033,12306,2010-01-04 21:00:00,42,0.460
413836034,11620,2008-08-14 19:30:00,39,0.450
413836035,1404,2010-07-12 22:30:00,45,0.492
413836036,6552,2009-08-13 04:30:00,9,0.014


In [75]:
df.ANON_ID = df.ANON_ID.astype(str)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 413836038 entries, 0 to 413836037
Data columns (total 4 columns):
 #   Column           Dtype         
---  ------           -----         
 0   ANON_ID          object        
 1   ADVANCEDATETIME  datetime64[ns]
 2   HH               int64         
 3   ELECKWH          float64       
dtypes: datetime64[ns](1), float64(1), int64(1), object(1)
memory usage: 12.3+ GB


In [76]:
df.sort_values('ADVANCEDATETIME', inplace=True)
df

,ANON_ID,ADVANCEDATETIME,HH,ELECKWH
7154,11628,2008-01-08 00:00:00,0,0.500
7155,11628,2008-01-08 00:30:00,1,0.450
7156,11628,2008-01-08 01:00:00,2,0.450
7157,11628,2008-01-08 01:30:00,3,0.500
7158,11628,2008-01-08 02:00:00,4,0.450
...,...,...,...,...
413833103,3569,2010-09-30 23:30:00,47,0.250
413833102,10876,2010-09-30 23:30:00,47,0.537
413833101,280,2010-09-30 23:30:00,47,0.055
413833107,16092,2010-09-30 23:30:00,47,0.054


In [78]:
df.columns

Index(['ANON_ID', 'ADVANCEDATETIME', 'HH', 'ELECKWH'], dtype='object')

In [79]:
final_df = df.pivot_table(index='ADVANCEDATETIME', columns='ANON_ID', values='ELECKWH')
final_df

ANON_ID,1,10,100,10000,10001,10002,10003,10004,10005,10006,...,999,9990,9991,9993,9994,9995,9996,9997,9998,9999
ADVANCEDATETIME,,,,,,,,,,,,,,,,,,,,,
2008-01-08 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008-01-08 00:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008-01-08 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008-01-08 01:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008-01-08 02:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2010-09-30 21:30:00,NaN,0.406,0.341,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.127,0.27,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-09-30 22:00:00,NaN,0.325,0.350,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.062,0.22,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-09-30 22:30:00,NaN,0.277,0.328,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.064,0.22,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [80]:
final_df = final_df.reset_index()
final_df = final_df.rename_axis(None, axis=1)
final_df

,ADVANCEDATETIME,1,10,100,10000,10001,10002,10003,10004,10005,...,999,9990,9991,9993,9994,9995,9996,9997,9998,9999
0,2008-01-08 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2008-01-08 00:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2008-01-08 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2008-01-08 01:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2008-01-08 02:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47486,2010-09-30 21:30:00,NaN,0.406,0.341,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.127,0.27,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47487,2010-09-30 22:00:00,NaN,0.325,0.350,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.062,0.22,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47488,2010-09-30 22:30:00,NaN,0.277,0.328,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.064,0.22,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47489,2010-09-30 23:00:00,NaN,0.229,0.320,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.041,0.16,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [81]:
final_df.rename(columns={'ADVANCEDATETIME': 'Timestamp'}, inplace=True)
final_df

,Timestamp,1,10,100,10000,10001,10002,10003,10004,10005,...,999,9990,9991,9993,9994,9995,9996,9997,9998,9999
0,2008-01-08 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2008-01-08 00:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2008-01-08 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2008-01-08 01:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2008-01-08 02:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47486,2010-09-30 21:30:00,NaN,0.406,0.341,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.127,0.27,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47487,2010-09-30 22:00:00,NaN,0.325,0.350,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.062,0.22,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47488,2010-09-30 22:30:00,NaN,0.277,0.328,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.064,0.22,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47489,2010-09-30 23:00:00,NaN,0.229,0.320,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.041,0.16,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [82]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47491 entries, 0 to 47490
Columns: 14320 entries, Timestamp to 9999
dtypes: datetime64[ns](1), float64(14319)
memory usage: 5.1 GB


In [83]:
final_df.set_index('Timestamp', inplace=True)
final_df

,1,10,100,10000,10001,10002,10003,10004,10005,10006,...,999,9990,9991,9993,9994,9995,9996,9997,9998,9999
Timestamp,,,,,,,,,,,,,,,,,,,,,
2008-01-08 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008-01-08 00:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008-01-08 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008-01-08 01:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008-01-08 02:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2010-09-30 21:30:00,NaN,0.406,0.341,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.127,0.27,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-09-30 22:00:00,NaN,0.325,0.350,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.062,0.22,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-09-30 22:30:00,NaN,0.277,0.328,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.064,0.22,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [94]:
df_copy = final_df.copy()
df_copy

,1,10,100,10000,10001,10002,10003,10004,10005,10006,...,999,9990,9991,9993,9994,9995,9996,9997,9998,9999
Timestamp,,,,,,,,,,,,,,,,,,,,,
2008-01-08 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008-01-08 00:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008-01-08 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008-01-08 01:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008-01-08 02:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2010-09-30 21:30:00,NaN,0.406,0.341,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.127,0.27,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-09-30 22:00:00,NaN,0.325,0.350,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.062,0.22,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-09-30 22:30:00,NaN,0.277,0.328,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.064,0.22,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [96]:
df_copy.columns = df_copy.columns.astype(int)

In [98]:
df_copy.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 47491 entries, 2008-01-08 00:00:00 to 2010-09-30 23:30:00
Columns: 14319 entries, 1 to 9999
dtypes: float64(14319)
memory usage: 5.1 GB


In [102]:
df_copy = df_copy[sorted(df_copy.columns)]
df_copy.columns = df_copy.columns.astype(str)
df_copy

,1,2,3,4,5,6,7,8,9,10,...,16239,16240,16242,16243,16244,16245,16246,16247,16248,16249
Timestamp,,,,,,,,,,,,,,,,,,,,,
2008-01-08 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008-01-08 00:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008-01-08 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008-01-08 01:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008-01-08 02:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2010-09-30 21:30:00,NaN,0.012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.406,...,NaN,0.356,0.344,0.339,0.186,NaN,NaN,NaN,NaN,NaN
2010-09-30 22:00:00,NaN,0.023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.325,...,NaN,0.250,0.297,0.307,0.159,NaN,NaN,NaN,NaN,NaN
2010-09-30 22:30:00,NaN,0.014,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.277,...,NaN,0.228,0.318,0.239,0.106,NaN,NaN,NaN,NaN,NaN


In [103]:
hkk.to_csv("/home/user/New_Buildings_Datasets/New-Bs/UK-Smart-trials/processed/30min/UK-smart-trails-30min.csv")

In [104]:
final_df[['1', '2', '3', '10', '16240', '16242', '16243', '16244']]

,1,2,3,10,16240,16242,16243,16244
Timestamp,,,,,,,,
2008-01-08 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008-01-08 00:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008-01-08 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008-01-08 01:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008-01-08 02:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
2010-09-30 21:30:00,NaN,0.012,NaN,0.406,0.356,0.344,0.339,0.186
2010-09-30 22:00:00,NaN,0.023,NaN,0.325,0.250,0.297,0.307,0.159
2010-09-30 22:30:00,NaN,0.014,NaN,0.277,0.228,0.318,0.239,0.106


In [105]:
df_copy_1h = hkk.resample('1H').sum(min_count=1)
df_copy_1h

,1,2,3,4,5,6,7,8,9,10,...,16239,16240,16242,16243,16244,16245,16246,16247,16248,16249
Timestamp,,,,,,,,,,,,,,,,,,,,,
2008-01-08 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008-01-08 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008-01-08 02:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008-01-08 03:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008-01-08 04:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2010-09-30 19:00:00,NaN,0.027,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.800,...,NaN,0.979,0.932,0.860,0.475,NaN,NaN,NaN,NaN,NaN
2010-09-30 20:00:00,NaN,0.026,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.758,...,NaN,0.830,1.207,0.681,0.378,NaN,NaN,NaN,NaN,NaN
2010-09-30 21:00:00,NaN,0.031,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.745,...,NaN,0.847,0.678,0.720,0.365,NaN,NaN,NaN,NaN,NaN


In [106]:
df_copy_1h.to_csv("/home/user/New_Buildings_Datasets/New-Bs/UK-Smart-trials/processed/hourly/UK-smart-trails-1H.csv")

In [2]:
df = pd.read_csv("/home/user/New_Buildings_Datasets/New-Bs/UK-Smart-trials/processed/hourly/UK-smart-trails-1H.csv")
df

,Timestamp,1,2,3,4,5,6,7,8,9,...,16239,16240,16242,16243,16244,16245,16246,16247,16248,16249
0,2008-01-08 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2008-01-08 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2008-01-08 02:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2008-01-08 03:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2008-01-08 04:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23923,2010-09-30 19:00:00,NaN,0.027,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.979,0.932,0.860,0.475,NaN,NaN,NaN,NaN,NaN
23924,2010-09-30 20:00:00,NaN,0.026,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.830,1.207,0.681,0.378,NaN,NaN,NaN,NaN,NaN
23925,2010-09-30 21:00:00,NaN,0.031,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.847,0.678,0.720,0.365,NaN,NaN,NaN,NaN,NaN
23926,2010-09-30 22:00:00,NaN,0.037,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.478,0.615,0.546,0.265,NaN,NaN,NaN,NaN,NaN


In [3]:
df.set_index("Timestamp", inplace=True)

In [4]:
gh = {}
for id in df.columns:
    gh[id] = (df[id].isna().sum() / df.shape[0]) * 100
gh

{'1': 27.833500501504517,
 '2': 47.75158809762621,
 '3': 36.88147776663323,
 '4': 16.578903376797058,
 '5': 40.45051822133066,
 '6': 65.19558676028085,
 '7': 16.449348044132396,
 '8': 24.903878301571382,
 '9': 71.8530591775326,
 '10': 47.007689735874294,
 '11': 66.11501170177199,
 '12': 47.107990638582415,
 '13': 43.066700100300906,
 '14': 46.94082246740221,
 '15': 26.922433968572385,
 '16': 22.099632230023403,
 '17': 41.261283851554666,
 '18': 27.22751588097626,
 '19': 46.50200601805417,
 '20': 47.141424272818455,
 '21': 13.954363089267805,
 '22': 71.2846873955199,
 '23': 18.066700100300903,
 '24': 24.832831828819792,
 '25': 47.44232698094283,
 '26': 42.14727515880976,
 '27': 46.94082246740221,
 '28': 47.02440655299231,
 '29': 46.94082246740221,
 '30': 59.9172517552658,
 '31': 31.30224005349381,
 '33': 17.32280173854898,
 '34': 14.648110999665665,
 '36': 24.811935807422266,
 '38': 28.21798729521899,
 '39': 47.141424272818455,
 '40': 21.205282514209294,
 '41': 16.787863590772318,
 '42'

In [6]:
missing_df = pd.DataFrame(gh.items(), columns=['Buildings', 'Percentage'])
missing_df

,Buildings,Percentage
0,1,27.833501
1,2,47.751588
2,3,36.881478
3,4,16.578903
4,5,40.450518
...,...,...
14314,16245,28.531428
14315,16246,26.015547
14316,16247,17.652959
14317,16248,32.514209


In [7]:
missing_df['Missing values'] = missing_df['Buildings'].apply(lambda x: df[x].isna().sum())
missing_df

,Buildings,Percentage,Missing values
0,1,27.833501,6660
1,2,47.751588,11426
2,3,36.881478,8825
3,4,16.578903,3967
4,5,40.450518,9679
...,...,...,...
14314,16245,28.531428,6827
14315,16246,26.015547,6225
14316,16247,17.652959,4224
14317,16248,32.514209,7780


In [8]:
missing_df['Total values'] = [len(df)] * len(missing_df)
missing_df

,Buildings,Percentage,Missing values,Total values
0,1,27.833501,6660,23928
1,2,47.751588,11426,23928
2,3,36.881478,8825,23928
3,4,16.578903,3967,23928
4,5,40.450518,9679,23928
...,...,...,...,...
14314,16245,28.531428,6827,23928
14315,16246,26.015547,6225,23928
14316,16247,17.652959,4224,23928
14317,16248,32.514209,7780,23928


In [9]:
missing_df.to_csv('missing_values.csv', index=False)

In [10]:
missing_df.Percentage.mean()

39.41502703748684

In [11]:
missing_df[missing_df.Percentage > missing_df.Percentage.mean()].shape

(7148, 4)

In [4]:
df = pd.read_csv("/home/user/New_Buildings_Datasets/Residential/UK-Smart-trials/processed/hourly/UK-smart-trails-1H.csv")
df.set_index("Timestamp", inplace=True)
df

,1,2,3,4,5,6,7,8,9,10,...,16239,16240,16242,16243,16244,16245,16246,16247,16248,16249
Timestamp,,,,,,,,,,,,,,,,,,,,,
2008-01-08 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008-01-08 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008-01-08 02:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008-01-08 03:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008-01-08 04:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2010-09-30 19:00:00,NaN,0.027,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.800,...,NaN,0.979,0.932,0.860,0.475,NaN,NaN,NaN,NaN,NaN
2010-09-30 20:00:00,NaN,0.026,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.758,...,NaN,0.830,1.207,0.681,0.378,NaN,NaN,NaN,NaN,NaN
2010-09-30 21:00:00,NaN,0.031,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.745,...,NaN,0.847,0.678,0.720,0.365,NaN,NaN,NaN,NaN,NaN


In [5]:
df.to_parquet("/home/user/New_Buildings_Datasets/Residential/UK-Smart-trials/processed/hourly/UKST-1H.parquet")